In [97]:
%run data_plot.ipynb

In [98]:
'''
initialize the variables for storing possible routes and the 
best route for visualization and saving in a solutions file
'''
sim_anneal_data = {'best_path': {},
            'possible_paths': {}}
solutions = [{}]

In [99]:
# Define the objective function to be optimized
def generate_route(origin):
    """
    generate_route(origin)
    generates a random route of variable length from the start node to either of the
    base stations. This however is done while checking if the there is an end to end bandwidth
    rate from the origin the the base station. This is done iteratively until a good path is found
    origin: the start node from which a path is being established
    returns: the route with an end-to-end transmission rate > 0 and the transmission rate
    """
    chosen_destination = choice(np.array(list(x_y_data.keys())[-2:]), 1, replace=False)[0]
    node_indices = [loc for loc in x_y_data.keys() if loc != origin and loc != chosen_destination]
    route_cost = 0
    while route_cost == 0:
        # length of path (chromosome) is not fixed so generate a random length of path for each iteration
        path_length = randint(1, len(node_indices))
        # choose indices to be used as genes
        chosen_indices = choice(np.array(node_indices), path_length, replace=False)
        # prepend and append origin and destination respectively to the generated path
        route = np.append(origin, chosen_indices)
        route = np.append(route, chosen_destination)
        route_cost = bandwidth_latency_calculation(route)

    return route, route_cost

In [100]:
def bandwidth_latency_calculation(route):
    """
    bandwidth_latency_calc(routes)
    calculates the end-to-end transmission rate for the route provided
    the function also stores
    route: the path whose end-to-end transmission is being calculated
    returns: the absolute path cost of the route
    """
    global sim_anneal_data, solutions
    route_costs = []

    # var for storing all path bandwidths
    path_bandwidths = []
    path_latency = 0
    # we do -1 because we need to get to second last node where we will check cost between it and last node
    for i in range(len(route) - 1):
        # get path distance
        path_distance = calculate_distance(x_y_data[route[i]], x_y_data[route[i + 1]])
        # get bandwidth between node and its next node and append it to path bandwidths var
        path_bandwidth = convert_distance_bandwidth(path_distance)
        path_latency += node_to_node_latency
        path_bandwidths.append(path_bandwidth)
    # compute end-to-end bandwidth rate
    end_to_end_bandwidth = min(path_bandwidths)
    # step latency down to 0 if bandwidth is 0
    if end_to_end_bandwidth == 0:
        absolute_path_cost = 0
    else:
        absolute_path_cost = round(absolute_cost_function(end_to_end_bandwidth, path_latency), 3)
        if len(sim_anneal_data['best_path']) == 0:
            solutions[0] = {"source node": f"Node-{route[0]}"}
            routing_path = ''
            for idx, sensor in enumerate(route[1:]):
                routing_path += f"(Node-{sensor}, {path_bandwidths[idx]} Mbps),"
            solutions[0]["routing path"] = routing_path
            solutions[0]["end-to-end transmission rate"] = f"{end_to_end_bandwidth} Mbps"
            solutions[0]["end-to-end latency"] = f"{path_latency} ms"
            solutions[0]["absolute path cost"] = f"{absolute_path_cost}"
            sim_anneal_data['best_path'][str(list(route))] = {'end_to_end_bandwidth': end_to_end_bandwidth,
                                                 'path_latency': path_latency,
                                                 'absolute_cost': absolute_path_cost
                                                }
        else:
            best_path = sim_anneal_data['best_path'][list(sim_anneal_data['best_path'].keys())[0]]
            if absolute_path_cost > best_path['absolute_cost']:
                solutions[0] = {"source node": f"Node-{route[0]}"}
                routing_path = ''
                for idx, sensor in enumerate(route[1:]):
                    routing_path += f"(Node-{sensor}, {path_bandwidths[idx]} Mbps),"
                solutions[0]["routing path"] = routing_path
                solutions[0]["end-to-end transmission rate"] = f"{end_to_end_bandwidth} Mbps"
                solutions[0]["end-to-end latency"] = f"{path_latency} ms"
                solutions[0]["absolute path cost"] = f"{absolute_path_cost}"
                sim_anneal_data['possible_paths'][list(sim_anneal_data['best_path'].keys())[0]] = {
                                                 'end_to_end_bandwidth': best_path['end_to_end_bandwidth'],
                                                 'path_latency': best_path['path_latency'],
                                                 'absolute_cost': best_path['absolute_cost']
                                                }
                sim_anneal_data['best_path'].pop(list(sim_anneal_data['best_path'].keys())[0])
                sim_anneal_data['best_path'][str(list(route))] = {
                                                 'end_to_end_bandwidth': end_to_end_bandwidth,
                                                 'path_latency': path_latency,
                                                 'absolute_cost': absolute_path_cost
                                                }
            else:
                sim_anneal_data['possible_paths'][str(list(route))] = {
                                                 'end_to_end_bandwidth': end_to_end_bandwidth,
                                                 'path_latency': path_latency,
                                                 'absolute_cost': absolute_path_cost
                                                }

    return absolute_path_cost

In [101]:
def simulated_annealing(origin, init_temp=100, final_temp=0.1, cooling_rate=0.95):
    """
    simulated_annealing(origin, init_temp=100, final_temp=0.1, cooling_rate=0.95)
    the simulated annealing algorithm main method that gets the shortest path to 
    either of the base stations.
    origin: the start node (sensor) from which the search is being conducted
    init_temp: the start temperature defined for the simulated annealing algorithm (default: 100)
    final_temp: the final temperature defined for the simulated annealing algorithm (default: 0.1)
    cooling_rate: the temperature cooling rate for the algorithm for the algorithm (default: 0.95)
    returns: sim_anneal_data and solutions variables with updates values
    """
    # re-initialize the 'sim_anneal_data' and 'solutions' global variables every time the function is called
    global sim_anneal_data, solutions
    sim_anneal_data = {'best_path': {},
                       'possible_paths': {}}
    solutions = [{}]
    current_path, current_path_cost = generate_route(origin)     
    current_temp = init_temp
    i = 1
    # Iterate until the temperature is below the final temperature
    while current_temp > final_temp:
        path, path_cost = generate_route(origin)
        # Calculate the change in solution quality
        delta = path_cost - current_path_cost  
        if delta >= 0:
            # If the new path is better, accept it as the current path
            current_path = path
            current_path_cost = path_cost
            if i % 5 == 0:
                if delta < 1:
                    break
        else:
            # If the new path is worse, accept it with a certain probability
            probability = math.exp(-delta / current_temp)
            if random.random() < probability:
                current_path = path
                current_path_cost = path_cost
        # Decrease the temperature according to the cooling rate
        current_temp *= cooling_rate
        i += 1

    # Return the final solution and its quality, and the list of temperatures
    return sim_anneal_data, solutions

In [102]:
#simulated_annealing(43)